In [62]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm

In [63]:
from google.colab import files

uploaded = files.upload()

In [64]:
df = pd.read_csv("AMES_housing_Price.csv")

In [65]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [67]:
subset = ['MSSubClass', 'LotArea', 'OverallQual','OverallCond','TotalBsmtSF','1stFlrSF','2ndFlrSF','GarageArea', 'PoolArea', 'MiscVal','MoSold','SalePrice']
df[subset].head(30)

,MSSubClass,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GarageArea,PoolArea,MiscVal,MoSold,SalePrice
0,60,8450,7,5,856,856,854,548,0,0,2,208500
1,20,9600,6,8,1262,1262,0,460,0,0,5,181500
2,60,11250,7,5,920,920,866,608,0,0,9,223500
3,70,9550,7,5,756,961,756,642,0,0,2,140000
4,60,14260,8,5,1145,1145,1053,836,0,0,12,250000
5,50,14115,5,5,796,796,566,480,0,700,10,143000
6,20,10084,8,5,1686,1694,0,636,0,0,8,307000
7,60,10382,7,6,1107,1107,983,484,0,350,11,200000
8,50,6120,7,5,952,1022,752,468,0,0,4,129900
9,190,7420,5,6,991,1077,0,205,0,0,1,118000


In [68]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)

In [69]:
numeric_df = df.select_dtypes(include=np.number).dropna()
display(numeric_df)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,6,5,1999,2000,0.0,0,0,953,953,953,694,0,1647,0,0,2,1,3,1,7,1,1999.0,2,460,0,40,0,0,0,0,0,8,2007,175000
1456,1457,20,85.0,13175,6,6,1978,1988,119.0,790,163,589,1542,2073,0,0,2073,1,0,2,0,3,1,7,2,1978.0,2,500,349,0,0,0,0,0,0,2,2010,210000
1457,1458,70,66.0,9042,7,9,1941,2006,0.0,275,0,877,1152,1188,1152,0,2340,0,0,2,0,4,1,9,2,1941.0,1,252,0,60,0,0,0,0,2500,5,2010,266500
1458,1459,20,68.0,9717,5,6,1950,1996,0.0,49,1029,0,1078,1078,0,0,1078,1,0,1,0,2,1,5,0,1950.0,1,240,366,0,112,0,0,0,0,4,2010,142125


In [70]:
df.isnull().sum()

,0
Id,0
MSSubClass,0
MSZoning,0
LotFrontage,259
LotArea,0
...,...
MoSold,0
YrSold,0
SaleType,0
SaleCondition,0


In [72]:
#predictors = ['MSSubClass', 'LotArea', 'OverallQual','OverallCond','TotalBsmtSF','1stFlrSF','2ndFlrSF','GarageArea', 'PoolArea', 'MiscVal','MoSold']
# wynik RMSE: 38174
#predictors = ['LotArea', 'OverallQual','OverallCond','TotalBsmtSF','1stFlrSF','2ndFlrSF','GarageArea', 'PoolArea']
# wynik RMSE: 38442
# predictors = ['LotArea','TotalBsmtSF','1stFlrSF','2ndFlrSF','GarageArea', 'PoolArea']
# wynik RMSE: 45768
# predictors = ['MSSubClass', 'LotArea', 'OverallQual','OverallCond','BsmtFinSF2','TotalBsmtSF','1stFlrSF','2ndFlrSF','GarageArea', 'PoolArea','GrLivArea','BsmtFullBath','BsmtHalfBath', 'MiscVal','MoSold',]
# wynik RMSE: 37319
predictors = ['MSSubClass', 'LotArea', 'OverallQual','OverallCond','BsmtFinSF2','TotalBsmtSF','1stFlrSF','2ndFlrSF','GarageArea', 'PoolArea','GrLivArea','BsmtFullBath','BsmtHalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd', 'MiscVal','MoSold',]
# wynik RMSE: 36629

outcome = 'SalePrice'
house_lm = LinearRegression()
house_lm.fit(df[predictors], df[outcome])

LinearRegression()

In [73]:
print(f'Intercept: {house_lm.intercept_:.3f}')
print('Coefficients:')
for name, coef in zip(predictors, house_lm.coef_):
    print(f' {name}: {coef}')

Intercept: -75763.712
Coefficients:
 MSSubClass: -128.22837385639005
 LotArea: 0.4210460572279634
 OverallQual: 23873.914099709404
 OverallCond: 1783.753585506851
 BsmtFinSF2: -10.067157642148686
 TotalBsmtSF: 15.301348508270035
 1stFlrSF: 67.83051502214143
 2ndFlrSF: 64.82109429795287
 GarageArea: 44.69236523417921
 PoolArea: -41.81954338803271
 GrLivArea: -14.409945157178521
 BsmtFullBath: 16017.3891890115
 BsmtHalfBath: 8688.193060974498
 BedroomAbvGr: -10759.956895496725
 KitchenAbvGr: -21130.942688019786
 TotRmsAbvGrd: 5207.875198489897
 MiscVal: -0.2219678426571114
 MoSold: -86.52096919940405


In [74]:
fitted = house_lm.predict(df[predictors])
RMSE = np.sqrt(mean_squared_error(df[outcome], fitted))
r2 = r2_score(df[outcome], fitted)
print(f'RMSE: {RMSE:.0f}')
print(f'r2: {r2:.4f}')

RMSE: 36629
r2: 0.7873


In [75]:
import statsmodels.api as sm

X = df[predictors]
X = sm.add_constant(X)
y = df[outcome]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.787
Model:                            OLS   Adj. R-squared:                  0.785
Method:                 Least Squares   F-statistic:                     296.3
Date:                Thu, 20 Nov 2025   Prob (F-statistic):               0.00
Time:                        11:12:31   Log-Likelihood:                -17414.
No. Observations:                1460   AIC:                         3.487e+04
Df Residuals:                    1441   BIC:                         3.497e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -7.576e+04   9957.857     -7.608      0.000   -9.53e+04   -5.62e+04
MSSubClass    -128.2284     27.532     -4.657      0.000    -182.236     -74.221
LotArea          0.4210      0.105      4.013      0.000       0.215       0.627
OverallQual   2.387e+04   1090.504     21.893      0.000    2.17e+04     2.6e+04
OverallCond   1783.7536    902.480      1.977      0.048      13.439    3554.068
BsmtFinSF2     -10.0672      6.186     -1.627      0.104     -22.202       2.067
TotalBsmtSF     15.3013      4.246      3.604      0.000       6.973      23.630
1stFlrSF        67.8305     20.886      3.248      0.001      26.860     108.801
2ndFlrSF        64.8211     20.450      3.170      0.002      24.707     104.935
GarageArea      44.6924      5.901      7.574      0.000      33.117      56.267
PoolArea       -41.8195     24.711     -1.692      0.091     -90.292       6.653
GrLivArea      -14.4099     20.392     -0.707      0.480     -54.411      25.591
BsmtFullBath  1.602e+04   2071.725      7.731      0.000     1.2e+04    2.01e+04
BsmtHalfBath  8688.1931   4177.132      2.080      0.038     494.283    1.69e+04
BedroomAbvGr -1.076e+04   1740.511     -6.182      0.000   -1.42e+04   -7345.750
KitchenAbvGr -2.113e+04   5321.267     -3.971      0.000   -3.16e+04   -1.07e+04
TotRmsAbvGrd  5207.8752   1298.303      4.011      0.000    2661.108    7754.642
MiscVal         -0.2220      1.962     -0.113      0.910      -4.071       3.627
MoSold         -86.5210    359.847     -0.240      0.810    -792.402     619.360
==============================================================================
Omnibus:                      640.795   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            90058.867
Skew:                          -0.983   Prob(JB):                         0.00
Kurtosis:                      41.426   Cond. No.                     1.57e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.57e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [76]:
X = df[predictors]
y = df[outcome]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_train = LinearRegression()
model_train.fit(X_train, y_train)

rmse_train = np.sqrt(mean_squared_error(y_train, model_train.predict(X_train)))
rmse_test = np.sqrt(mean_squared_error(y_test, model_train.predict(X_test)))

print(f'RMSE training: {rmse_train:.0f}')
print(f'RMSE test: {rmse_test:.0f}')

RMSE training: 36349
RMSE test: 38354


In [77]:
from sklearn.model_selection import cross_validate

# k-Fold CV z pełnymi statystykami
cv_results = cross_validate(LinearRegression(), X, y, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)

test_scores = np.sqrt(-cv_results['test_score'])
train_scores = np.sqrt(-cv_results['train_score'])

print("RMSE dla każdego fold'u:")
for i, (train, test) in enumerate(zip(train_scores, test_scores)):
    print(f'Fold {i+1}: train={train:.0f}, test={test:.0f}')

print(f'\nŚrednia test RMSE: {test_scores.mean():.0f}')
print(f'Odchylenie std: {test_scores.std():.0f}')

RMSE dla każdego fold'u:
Fold 1: train=38209, test=29916
Fold 2: train=36843, test=36111
Fold 3: train=36174, test=39256
Fold 4: train=37683, test=32323
Fold 5: train=33277, test=49439

Średnia test RMSE: 37409
Odchylenie std: 6809
